<a href="https://colab.research.google.com/github/soundarya3333/Diabetes-data-analysis/blob/main/Integrated_NER_and_Summarization_Processor_python_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers beautifulsoup4 requests nltk


In [ ]:
import nltk
import re
import heapq
import numpy as np
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import requests
from bs4 import BeautifulSoup

# Download necessary NLTK packages
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Function for Extractive Summarization using TextRank
def extractive_summarization(text, num_sentences=3):
    sentences = nltk.sent_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')

    word_frequencies = {}
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords and word.isalnum():
            word_frequencies[word.lower()] = word_frequencies.get(word.lower(), 0) + 1

    max_freq = max(word_frequencies.values())
    for word in word_frequencies:
        word_frequencies[word] /= max_freq

    sentence_scores = {}
    for sent in sentences:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies:
                sentence_scores[sent] = sentence_scores.get(sent, 0) + word_frequencies[word]

    summarized_sentences = heapq.nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    return ' '.join(summarized_sentences)

# Function for Abstractive Summarization using T5 Transformer
def abstractive_summarization(text, model, tokenizer, max_length=150):
    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
# Function to summarize from a text file
def summarize_from_file(file_path, model, tokenizer, method='extractive', num_sentences=3):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    if method == 'extractive':
        return extractive_summarization(text, num_sentences)
    elif method == 'abstractive':
        return abstractive_summarization(text, model, tokenizer)
    else:
        raise ValueError("Invalid summarization method. Choose 'extractive' or 'abstractive'.")

# Function to summarize a URL article
def summarize_from_url(url, model, tokenizer, method='extractive', num_sentences=3):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')
    text = ' '.join([para.get_text() for para in paragraphs])

    if method == 'extractive':
        return extractive_summarization(text, num_sentences)
    elif method == 'abstractive':
        return abstractive_summarization(text, model, tokenizer)
    else:
        raise ValueError("Invalid summarization method. Choose 'extractive' or 'abstractive'.")


In [ ]:
# Load Pre-trained T5 Model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Download necessary NLTK packages
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Function for Extractive Summarization using TextRank
def extractive_summarization(text, num_sentences=3):
    sentences = sent_tokenize(text)
    stopwords_list = stopwords.words('english')

    word_frequencies = {}
    for word in word_tokenize(text):
        if word.lower() not in stopwords_list and word.isalnum():
            word_frequencies[word.lower()] = word_frequencies.get(word.lower(), 0) + 1

    max_freq = max(word_frequencies.values(), default=1)
    for word in word_frequencies:
        word_frequencies[word] /= max_freq

    sentence_scores = {}
    for sent in sentences:
        for word in word_tokenize(sent.lower()):
            if word in word_frequencies:
                sentence_scores[sent] = sentence_scores.get(sent, 0) + word_frequencies[word]

    summarized_sentences = heapq.nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    return ' '.join(summarized_sentences)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import re
from collections import Counter
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

def extractive_summarization(text, num_sentences=3):
    # Split text into sentences using regex
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Tokenize words and remove punctuation
    words = re.findall(r'\b\w+\b', text.lower())

    # Remove common stopwords
    stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'}
    words = [word for word in words if word not in stopwords]

    # Calculate word frequencies
    word_freq = Counter(words)

    # Score sentences
    sentence_scores = {}
    for sentence in sentences:
        # Calculate score based on word frequencies
        score = sum(word_freq[word.lower()] for word in re.findall(r'\b\w+\b', sentence.lower()) if word not in stopwords)
        sentence_scores[sentence] = score

    # Select top sentences
    top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]

    # Preserve original order
    summary_sentences = [sent for sent in sentences if sent in top_sentences]

    return ' '.join(summary_sentences)

def abstractive_summarization(text, model, tokenizer):
    # Prepare input for T5 model
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary
    outputs = model.generate(inputs, max_length=150, num_beams=4, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return summary

# Load the pre-trained model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Example usage
text = """
Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data. The goal is a computer capable of understanding the contents of documents, including the contextual nuances of the language within them. The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.

Challenges in natural language processing frequently involve speech recognition, natural language understanding, and natural-language generation. Based on long-standing goals in the field, NLP tasks include: machine translation, textual entailment, discourse analysis, text summarization, question answering, and natural language queries to databases.
"""

print("Extractive Summary:")
print(extractive_summarization(text))

print("\nAbstractive Summary:")
print(abstractive_summarization(text, model, tokenizer))

Extractive Summary:

Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data. Challenges in natural language processing frequently involve speech recognition, natural language understanding, and natural-language generation. Based on long-standing goals in the field, NLP tasks include: machine translation, textual entailment, discourse analysis, text summarization, question answering, and natural language queries to databases.

Abstractive Summary:
natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence. the goal is a computer capable of understanding the contents of documents. the technology can then accurately extract information and insights contained in the documents as well as categorize and organize the doc

In [ ]:
import spacy
import json
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

class AdvancedTextProcessor:
    def __init__(self, ner_model='en_core_web_sm', summarizer_model='t5-small'):
        """
        Initialize the advanced text processor with NER and summarization capabilities.

        Supported models:
        - NER: en_core_web_sm (default, lighter), en_core_web_md (medium), en_core_web_lg (large)
        - Summarizer: t5-small (lightweight)
        """
        # Initialize NER model (spaCy)
        try:
            self.nlp = spacy.load(ner_model)
        except OSError:
            print(f"Downloading {ner_model} model...")
            spacy.cli.download(ner_model)
            self.nlp = spacy.load(ner_model)

        # Initialize summarization model (T5)
        self.tokenizer = T5Tokenizer.from_pretrained(summarizer_model)
        self.model = T5ForConditionalGeneration.from_pretrained(summarizer_model)

    def extract_entities(self, text):
        """
        Extract and categorize named entities with advanced metadata
        """
        doc = self.nlp(text)

        # Structured entity extraction
        entities = []
        for ent in doc.ents:
            entities.append({
                'text': ent.text,
                'label': ent.label_,
                'start_char': ent.start_char,
                'end_char': ent.end_char
            })

        return entities

    def entity_relationship_graph(self, text):
        """
        Create a relationship graph between named entities
        """
        doc = self.nlp(text)

        # Create graph
        G = nx.Graph()

        # Track entity mentions
        entity_mentions = defaultdict(list)
        for ent in doc.ents:
            entity_mentions[ent.label_].append(ent.text)

        # Add nodes and edges
        for label, entities in entity_mentions.items():
            for entity in entities:
                G.add_node(entity, type=label)

        # Simple co-occurrence as basic relationship
        for sent in doc.sents:
            sent_entities = [ent.text for ent in sent.ents]
            for i in range(len(sent_entities)):
                for j in range(i+1, len(sent_entities)):
                    if sent_entities[i] != sent_entities[j]:
                        G.add_edge(sent_entities[i], sent_entities[j])

        return G

    def visualize_entity_graph(self, graph):
        """
        Visualize the entity relationship graph
        """
        plt.figure(figsize=(12, 8))
        pos = nx.spring_layout(graph, k=0.5)  # positions for all nodes

        # Draw nodes with color based on entity type
        node_colors = [self._get_node_color(graph.nodes[node]['type']) for node in graph.nodes()]
        nx.draw(graph, pos, with_labels=True, node_color=node_colors,
                node_size=1000, font_size=8, font_weight="bold")

        plt.title("Entity Relationship Graph")
        plt.tight_layout()
        plt.savefig('entity_graph.png')
        plt.close()

    def _get_node_color(self, entity_type):
        """
        Assign colors to different entity types
        """
        color_map = {
            'PERSON': '#FF6B6B',     # Coral Red
            'ORG': '#4ECDC4',         # Teal
            'GPE': '#45B7D1',         # Sky Blue
            'LOC': '#2AB673',         # Green
            'DATE': '#FFA07A',        # Light Salmon
            'MONEY': '#FFD700',       # Gold
            'CARDINAL': '#9370DB'     # Medium Purple
        }
        return color_map.get(entity_type, '#A9A9A9')  # Default to Dark Gray

    def generate_entity_report(self, text):
        """
        Generate a comprehensive report of named entities
        """
        doc = self.nlp(text)

        # Aggregate entities by type
        entity_summary = defaultdict(list)
        for ent in doc.ents:
            entity_summary[ent.label_].append(ent.text)

        # Convert to JSON for easy parsing
        return json.dumps({
            'total_entities': len(doc.ents),
            'entity_types': dict(entity_summary)
        }, indent=2)

    def extractive_summarization(self, text, num_sentences=3):
        # Split text into sentences using regex
        sentences = re.split(r'(?<=[.!?])\s+', text)

        # Tokenize words and remove punctuation
        words = re.findall(r'\b\w+\b', text.lower())

        # Remove common stopwords
        stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'with', 'by'}
        words = [word for word in words if word not in stopwords]

        # Calculate word frequencies
        word_freq = Counter(words)

        # Score sentences
        sentence_scores = {}
        for sentence in sentences:
            # Calculate score based on word frequencies
            score = sum(word_freq[word.lower()] for word in re.findall(r'\b\w+\b', sentence.lower()) if word not in stopwords)
            sentence_scores[sentence] = score

        # Select top sentences
        top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]

        # Preserve original order
        summary_sentences = [sent for sent in sentences if sent in top_sentences]

        return ' '.join(summary_sentences)

    def abstractive_summarization(self, text):
        # Prepare input for T5 model
        inputs = self.tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

        # Generate summary
        outputs = self.model.generate(inputs, max_length=150, num_beams=4, early_stopping=True)

        # Decode the summary
        summary = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        return summary

    def process_text(self, text, num_summary_sentences=3):
        """
        Combine NER extraction and summarization into one process
        """
        # Extract named entities
        entities = self.extract_entities(text)

        # Generate entity relationship graph
        graph = self.entity_relationship_graph(text)

        # Generate summaries
        extractive_summary = self.extractive_summarization(text, num_summary_sentences)
        abstractive_summary = self.abstractive_summarization(text)

        # Generate entity report
        entity_report = self.generate_entity_report(text)

        # Return the results
        return {
            'entities': entities,
            'entity_graph': graph,
            'extractive_summary': extractive_summary,
            'abstractive_summary': abstractive_summary,
            'entity_report': json.loads(entity_report)
        }

# Example Usage
def main():
    text = """
    Apple Inc. was founded by Steve Jobs in Cupertino, California on April 1, 1976. The company has generated over $260 billion in revenue.
    Google's headquarters is located in Mountain View, California. The company was founded by Larry Page and Sergey Brin in 1998.
    """

    processor = AdvancedTextProcessor()

    results = processor.process_text(text)

    # Print results
    print("Named Entities:")
    print(json.dumps(results['entities'], indent=2))

    print("\nEntity Relationship Report:")
    print(results['entity_report'])

    print("\nExtractive Summary:")
    print(results['extractive_summary'])

    print("\nAbstractive Summary:")
    print(results['abstractive_summary'])

    # Visualize the entity relationship graph
    processor.visualize_entity_graph(results['entity_graph'])

if __name__ == "__main__":
    main()


Named Entities:
[
  {
    "text": "Apple Inc.",
    "label": "ORG",
    "start_char": 5,
    "end_char": 15
  },
  {
    "text": "Steve Jobs",
    "label": "PERSON",
    "start_char": 31,
    "end_char": 41
  },
  {
    "text": "Cupertino",
    "label": "GPE",
    "start_char": 45,
    "end_char": 54
  },
  {
    "text": "California",
    "label": "GPE",
    "start_char": 56,
    "end_char": 66
  },
  {
    "text": "April 1, 1976",
    "label": "DATE",
    "start_char": 70,
    "end_char": 83
  },
  {
    "text": "$260 billion",
    "label": "MONEY",
    "start_char": 116,
    "end_char": 128
  },
  {
    "text": "Google",
    "label": "ORG",
    "start_char": 145,
    "end_char": 151
  },
  {
    "text": "Mountain View",
    "label": "GPE",
    "start_char": 181,
    "end_char": 194
  },
  {
    "text": "California",
    "label": "GPE",
    "start_char": 196,
    "end_char": 206
  },
  {
    "text": "Larry Page",
    "label": "PERSON",
    "start_char": 235,
    "end_char": 245
  },
 

<ipython-input-17-78a660523e01>:91: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
